$$
\def\E{\mathbb{E}}
\def\1{\mathbb{1}}
$$
In questo modello multiperiodale un'azienda fallisce quando alla scadenza del proprio debito il valore degli attivi si trova al di sotto di una soglia fissa K. Nel caso in cui si trovi sopra l'azienda può accedere ad un nuovo finziamento, ovviamente ad uno spread maggiore del precedente qualora gli attivi siano scesi (minore se saliti).
L'azienda rinnova i propri finanziamenti ogni $\tau$ anni, in teoria per sempre. Per motivi numerici è opportuno inserire una scadenza $T >> \tau.$
Ad ogni data di rinnovo $t = i \cdot \tau$ (dove i = 0,...,N), i finanziatori dell'azienda determinano lo spread minimo a cui sono dispositi a prestare soldi dalla seguente relazione:
$$
L(t) = \E\big[L(t)\cdot(1+s(t))\1_{\{A(t+\tau) > K\}} + A(t+\tau)\1_{\{A(t+\tau < K\}}
|\mathcal{F}_t\big]
$$
Lo spread si può quindi determinare come:
$$
s(t) = \frac{1}{L \cdot N(d_2)}\big[ L(t)\cdot \big(\1-N(d_2)\big) - A(t)\cdot\big(1-N(d_1)\big)\big]
$$
dove $L(t)$ è calcolabile partendo dal finanziamento originario in $t = 0$ come:
$$
L(t=i\tau) = \prod_{j=0}^{j=i-1}(1+s(j\tau))L(0)
$$
La valutazione dell'equity potrebbe essere quindi effettuata in modalità forward (ad esempio tramite montecarlo) fino alla scadenza fittizia T attualizzando:
* 0, nel caso in cui esista un A(t) < K
* $max(A(T) - L(T), 0)$ se arriva a scadenza

Questo modello ha una cosenguenza, ovvero che il default non dipende dalla storia del passivo (solo il payoff ne dipende). Vederemo in seguito se possibile cambiare qualche assunzione.


In [11]:
A = 100

In [12]:
K = 50

In [13]:
vol = 0.2

In [14]:
tau = 1.0

In [15]:
L = 90.0

In [16]:
T = 50

In [17]:
import numpy as np
import scipy as sp
import scipy.stats
import math

In [18]:
def s(At, Lt):
    d1 = (math.log(At / K) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / K) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

In [19]:
print s(A,L)

0.000180191679296


In [20]:
def pathGen(runs):
    np.random.seed(1)
    nsteps = int((T / tau) + 1)
    vol2 = vol * vol
    epsilon = np.random.normal(0, 1, (runs, nsteps - 1))
    paths = np.ones((nsteps, runs))
    paths[0,:] = A
    for i in range(nsteps)[1:]:
        paths[i,:] = paths[i-1,:] * np.exp(-0.5*vol2 * tau + vol * math.sqrt(tau) * epsilon[:,i-1])
    return paths

In [21]:
def payoff(path):
    np.seterr('raise')
    # Check for default
    if min(path) < K:
        return 0.
    
    # Compute the spread
    try:
        Lt = L
        for i in range(len(path) - 1):
            spread = s(path[i], Lt)
            Lt = Lt * (1 + spread)
    except:
        return 0
    
    return max(path[-1] - Lt,0)

In [158]:
runs = 10000


In [22]:
def E():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff(paths[:,i])
    return payoffs.mean()
        

In [160]:
print E()

43.4665855053


Questo modello non funziona, dovremmo ottenere un numero vicino a A - L iniziali. Ci sono più motivi:
* è difficile capire qual è il payoff a scadenza: ho messo il max tra A - L e 0 poiché l'equity non può diventare negativa, ma così facendo il problema non è ben specificato. Ci sono path che terminano con A vicino a K, $L(T)$ enorme e nessuno che soffre A - L. Anche togliendo il max dalla condizione finale non sembra risolvere
* L può crescere indiscriminatamente senza portare al fallimento, con problemi (numerici e concettuali) enormi.

E' necessario cambiare le specifiche del modello, inserendo una condizione di fallimento differente Stavo pensando di inserire una condizione sullo spread, ma credo che questo porti alla necessità di risolvere il problema backward.
Come primo tentativo per ovviare a questo problema si può ipotizzare come condizione di fallimento quella in cui gli attivi si trovano al di sotto di una percentuale $X$ dei passivi emessi alla data precedente.

In [161]:
X = 0.4

In [23]:
def s_perc(At, Lt):
    d1 = (math.log(At / (Lt * X)) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / (Lt * X)) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

Il problema, come nel caso precedente, è quello di trovare un payoff terminale adeguato. Probabilmente la cosa migliore è ipotizzare, per quanto assurdo, che all'ultimo periodo valga il modello uniperiodale alla Merton e valutare la condizione terminale a $T - \tau$ con il valore dell'equity dato dal valore della call. Rispetto a come ho tratto il caso precedente devo quindi inserire un ulteriore funzione per la deteriminazione dello spread e la funzione di pricing dell'equity.

In [24]:
def bs(A, L):
    d1 = (math.log(A / L) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(A / L) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    call_price = A * sp.stats.norm.cdf(d1) - L * sp.stats.norm.cdf(d2)   
    put_price = L * sp.stats.norm.cdf(-d2) - A * sp.stats.norm.cdf(-d1)
    return put_price, call_price

In [25]:
def b1(A, L, s):
    put, call = bs(A, L * (1+s))
    return L * (1 + s) - put

In [26]:
def s_merton(A, L):
    res = sp.optimize.minimize(
    lambda s: (b1(A, L, s) - L) ** 2,
    [0.0],
    bounds=[(-0.1, 1.0)])
    return res.x[0]


In [27]:
def e_merton(A, L, s):
    put, call = bs(A, L * (1+s))
    return call

In [28]:
def payoff_perc(path):
    Lt = L
    for i in range(len(path) - 1):
        if path[i+1] < X * Lt:
            # Default
            return 0
        if i == (len(path) - 2):
            spread = s_merton(path[i], Lt)
            return e_merton(path[i], Lt, spread)
        else:
            spread = s_perc(path[i], Lt)

        Lt = Lt * (1 + spread)
    
    return max(path[-1] - Lt,0)

In [168]:
def E_perc():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff_perc(paths[:,i])
    return payoffs.mean()

In [169]:
print E_perc()

48.0665248717


Anche questo modello non funziona. Il problema credo dipenda dal fatto che in realtà queste specifiche non sono corrette. Ad esempio nel modello di Merton base il caso in cui il forward degli asset è inferiore al passivo non è ammissibile: non esiste nessun ente disposto a finanziare questo tipo di aziena. Sotto l'esempio numerico di un'aziena che ha debito per 100 e forward degli asset parti a 99.9.

In [29]:
print s_merton(99.9, 100)

1.0


Ovvero lo spread massimo possibile del risolutore numerico pari al 100%. Quindi, ipotizzare che valga Merton alla scadenza non è opportuno dato che il valore dell'attivo può scendere al di sotto del passivo. Anzi, la possibilità di un rifinanziamento tramite la sola emissione di nuovi bond in questi casi non è mai verosimile: c'è evidentemente da affrontre il modello in modo molto differente.
Mi vengono in mente due possibilità:

1. La condizione di fallimento, anche in modello multiperiodale, è quella in cui gli attivi si trovino al di sotto delle liabilities da ripagare.
2. Nel caso in cui gli attivi siano al di sotto delle liabilities ma sopra lo il livello di fallimento (ad es. $X \cdot L$), viene effettuato un aumento di capitale. Rimane il problema di cosa succede all'ultimo periodo.

Direi di procedere con la 1. Entrambe i casi possono essere affrontati in due modi differenti:
1. con un rifinanziamento periodico tramite passivo caratterizzato da un'unica scadenza (come assunto finora)
2. ipotizzando che vi sia una struttura a termine del passivo che viene di volta in volta rollata. 

Si procede con 1.
In questo modello, ai fini della valutazione dell'equity e dei bond/spread, dovrebbe essere rilevante solo il primo periodo. Adesso lo verifico empiricamente...


In [30]:
def payoff_multi_merton_1(path):
    Lt = L
    for i in range(len(path) - 1):
        spread = s_merton(path[i], Lt)
        Lt = Lt * (1 + spread)
        if path[i+1] < Lt:
            return 0
        if i == (len(path) - 2):
            return max(path[-1] - Lt,0)


In [31]:
def E_multi_merton_1():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff_multi_merton_1(paths[:,i])
    return payoffs.mean()

In [173]:
T = 20

In [174]:
print E_multi_merton_1()

10.7570490754


Direi che ci siamo (ci si aspettava un valore prossimo a $A(0) - L(0)$). Se il valore dell'equity è determinato univocamente dal primo periodo, a cosa ci serve un modello multiperiodale? Uno degli obiettivi è quello di capire come varia l'equity quando viene incrementato l'attivo mediante emissione di nuovo finanziamento (cambiando la rischiosità dell'attivo). A titolo di esempio prendiamo il caso in cui l'aziena acquisti, per un ammontare C, un bond risk free con scadenza pari a $\tau$ e rendimento nullo. L'equity prima di questa operazione vale evidentemente $A(0) - L(0)$ e su L viene pagato lo spread di Merton. 
Dopo questa operazione l'equity cambia valore:
$$
E_2 = \E[ \max(0, A(T) + C - L(1+s) - C(1+s')) ]
$$
dove s è lo spread prima dell'operazione, s' quello successivo, per determinare il quale si deve risolvere la seguente equazione:
$$
\begin{align}
C &= \E[ \min(C(1+s'),  (A(T) + C) \frac{C}{L+C}) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C}, A(T) + C) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C} - C, A(T)) + C ] \\
&= \frac{C}{L+C} \E[ \min((1+s')(L+C) - C, A(T)) ] + \frac{C^2}{L+C} \\
\end{align}
$$
dove è stato ipotizzato un recovery proporzionale al face value del debito.

In [32]:
def C_price(A, L, C, sprime):
    strike = (L + C) * (1 + sprime) - C
    put, call = bs(A, strike)
    return C / (L + C) * (strike - put) + (C ** 2) / (L + C)

In [33]:
def e2(A, L, s, sprime):
    put, call = bs(A, L * (1 + s) + C * sprime)
    return call

In [34]:
def sprime(A, L, C):
    res = sp.optimize.minimize(
    lambda s: (C_price(A, L, C, s) - C) ** 2,
    [0.0],
    bounds=[(-0.5, 1.0)])
    return res.x[0]


In [35]:
s = s_merton(A, L)

In [36]:
s

0.066005296966200763

In [37]:
C=20

In [38]:
s_prime = sprime(A, L, C)

In [39]:
s_prime

0.054004334796322315

In [40]:
e2(A, L, s, s_prime)

9.4258825881647326

Quindi l'equity diminuisce. Adesso vediamo cosa succede nell'ipotesi in cui il bond acquistato C abbia una maturity pari a 2 $\tau$. A $t = \tau$ deve essere rifinanziato (nel caso in cui non ci sia fallimento), un importo pari a $L(1+s) + C(1+s_{prime})$ ad un nuovo spread $s_2$. Il nuovo spread soddisfa la seguente equazione:
$$
\begin{align}
L(1+s) + C(1+s_{prime}) = L_{tot}  &= \E\big[\min(L_{tot} \cdot (1 + s_2), A(2\tau) + C)\big]  \\
 & = \E\big[\min(L_{tot} \cdot (1 + s_2) - C, A(2\tau))\big] + C
\end{align}
$$

In [62]:
def C2_price(A, Ltot, C, s2):
    strike = Ltot * (1 + s2) - C
    put, call = bs(A, strike)
    return (strike - put) + C

def s2(A, Ltot, C):
    res = sp.optimize.minimize(
    lambda s: (C2_price(A, Ltot, C, s) - Ltot) ** 2,
    [0.0],
    bounds=[(-0.5, 1.0)])
    return res.x[0]

T = 1.
runs = 1
A = 100.0
L = 90.0
C = 20.0
vol = 0.1
# Prima proviamo con un singolo finanziamento lungo tau = 2
print "Bond con maturity = tau"
tau = 1.
s = s_merton(A, L)
s_prime = sprime(A, L, C)
print s, s_prime, e2(A, L, s, s)

print "Bond con maturity = 2*tau"
print "Single financing"
tau = 2.
s = s_merton(A, L)
s_prime = sprime(A, L, C)
print s, s_prime, e2(A, L, s, s_prime)
# Ora con 2 finanziamenti
print "Rolling financing"
tau = 1.0
paths = pathGen(runs)
payoffs = np.zeros(runs)
s = s_merton(A, L)
s_prime = sprime(A, L, C)
totL1 = L*(1+s) + C*(1+s_prime)

for i in range(runs):
    if (paths[1, i] + C) < totL1:
         payoffs[i] = 0.
    else:
        # Find the new spread
        s_2 = s2(paths[1, i], totL1, C)
        payoffs[i] = max(paths[2, i] + C - totL1 * (1 + s_2), 0.)
print s, s_prime, payoffs.mean()


Bond con maturity = tau
0.00953198697104 0.0077988978585 9.84453935938
Bond con maturity = 2*tau
Single financing
0.0274233096586 0.0224372485795 9.69515074734
Rolling financing


IndexError: index 2 is out of bounds for axis 0 with size 2

La diminuzione in valore dell'equity rollando il finanziamento è la medesima (a meno dell'errore montecarlo) che si ha nel caso in cui l'obbligazione C ha una maturity pari a $\tau$: dipende quindi non dalla scadenza del titolo, ma dalla frequenza di rifinanziamento. Questo si poteva evincere anche da quanto osservato prima relativamente al modello rolling di Merton: nel caso in esempio in cui la maturity è pari a $2\tau$, alla data di rifinanziamento $\tau$ il valore dell'equity sarà pari al $\max(A(\tau) + C - L\cdot(1 + s)(1+s'),0)$, esattamente lo stesso del caso in cui il bond dura $\tau$.

Domanda di intermezzo: dopo ogni periodo, se l'azienda ha prodotto utili (i.e. gli asset sono saliti), l'equity potrebbe essere remunerato liquidando parte degli attivi. Questo influisce o meno?

Intermezzo, cosa succede in un modello uniperiodale se un'azienda finanzia un incremento sull'espozione del proprio attivo?
$$
\begin{align}
E &= \E\big[\max\big(A(\tau)\cdot(1+\alpha) - L(1+s) - C(1+s'),0\big)\big] \\
&= (1+\alpha)BS(A, K=\frac{L(1+s) - C(1+s')}{1+\alpha})
\end{align}
$$
con $\alpha = C / A(0)$. Per determinare $s'$ si dovrebbe valutare lo spread fair che risolve
$$
$$
\begin{align}
C &= \E[ \min(C(1+s'),  (1+\alpha)A(T) \frac{C}{L+C}) ] \geq C \\
&= \frac{(1+\alpha)C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C(1+\alpha)}, A(T) ]
\end{align}
$$
$$

In [42]:
def E_reinvest(A, L, C, s, s_prime):
    alpha = C / A
    K = (L*(1+s) + C*(1+s_prime))/(1+alpha)
    put, call = bs(A, K)
    return call * (1+alpha)

In [43]:
def C_reinvest(A, L, C, s, s_prime):
    alpha = C / A
    strike = (L + C) * (1 + s_prime) / (C*(1+alpha))
    put, call = bs(A, strike)
    return(1 + alpha) * C / (L + C) * (strike - put)

In [44]:
def s_prime_reinvest(A, L, C, s):
    res = sp.optimize.minimize(
    lambda s_prime: (C_reinvest(A, L, C, s, s_prime) - C) ** 2,
    [0.0],
    bounds=[(-0.1, 1.0)])
    return res.x[0]
    

In [45]:
def E_reinvest_check(A, L, C, s, s_prime):
    alpha = C / A
    K = (L*(1+s) + C*(1+s_prime))
    put, call = bs(A*(1+alpha), K)
    return call 

A = 100.0
L = 90.0
C = 1.0
vol = 0.2
tau = 1.0
s = s_merton(A, L)
print s
print e_merton(A, L, s)
s_prime = s_prime_reinvest(A, L, C, s)
print s_prime
print E_reinvest(A, L, C, s, s_prime)
print E_reinvest(A, L, C, s, 0.)
print E_reinvest_check(A, L, C, s, 0.)

0.0660052969662
10.0000002428
0.0668616728674
10.0418026854
10.0779865126
10.0779865126


Proviamo a ragionare su cosa succede ad una azienda che ha 2 finanziamenti con scadenze differenti, $\tau$ e $2\tau$, importi $L1$ e $L2$. Il finanziatore con scadenza minore applicherà un fair spread $s1$ dato da:
$$
L1 = L1(1+s1)\cdot\E\big[\1\{A(\tau)>L1(1+s1)\}\big] +\frac{L1}{L1 + L2}\E\big[A(\tau)\cdot\1\{A(\tau)<L1(1+s1)\}\big]\big]
$$
La situazione è un po' più complicata per L2 poiché deve tener conto di:
- probabiilità congiunta di sopravvivenza a T1, T2;
- spread di rifinanziamento "forward" s12 per L1.

Ci sono 3 scenari:
1. $A(\tau) < L1(1+s1)$ -> $L2/(L1+L2)*A(\tau)$
2. $A(\tau) > L1(1+s1) \cap \big[A(2*\tau) < L1(1+s1)(1+s12) + L2(1+s2)\big]$ -> $L2/(L1+L2)*A(2*\tau)$
2. $A(\tau) > L1(1+s1) \cap \big[A(2*\tau) >= L1(1+s1)(1+s12) + L2(1+s2)\big]$ -> $L2*(1+s2)$

La formula di pricing si può scrivere come:
$$
L2 = \frac{L2}{L1+L2}\E\big[A(\tau)\cdot\1\{A(\tau)<L1(1+s1)\}\big] + \frac{L2}{L1+L2}\E\big[A(2*\tau)\1\big\{ A(\tau) > L1(1+s1) \cap \big[A(2*\tau) < L1(1+s1)(1+s12) + L2(1+s2)\big] \big\}\big] + L2(1+s2)E\big[\1\big\{ A(\tau) > L1(1+s2) \cap \big[A(2*\tau) >= L1(1+s1)(1+s12) + L2(1+s2)\big]\big\}\big]
$$

Per calcolare questa formula è necessario anche stimare s12 dal punto di vista del finanziatore L2. Qui si entra nella metafisica... Proviamo. Ci sono 3 scenari:
1. $A(\tau) < L1(1+s2)$ -> 0. Non c'è rifinanziamento
2. $A(\tau) > L1(1+s2) \cap \big[A(2*\tau) < L1(1+s1)(1+s12) + L2(1+s2)\big]$ -> $L1/(L1+L2)*A(2*\tau)$
3. $A(\tau) > L1(1+s2) \cap \big[A(2*\tau) >= L1(1+s1)(1+s12) + L2(1+s2)\big]$ -> $L1*(1+s1)*(1+s12)$

Vediamo come si scrive una ipotetica formula di pricing:
$$
L(1+s1) \E\big[A(\tau) > L1(1+s1) \big] = \frac{L1}{L1+L2}\E\big[A(\tau)\cdot\1\big\{ A(\tau) > L1(1+s1) \cap \big[A(2*\tau) < L1(1+s1)(1+s12) + L2(1+s2)\big] \big\}\big] + L1(1+s1)(1+s12)E\big[\1\big\{ A(\tau) > L1(1+s2) \cap \big[A(2*\tau) >= L1(1+s1)(1+s12) + L2(1+s2)\big]\big\}\big] \\
$$

A questo punto abbiamo 2 equazioni in 2 incognite che devono essere risolte contemporaneamente. Vediamo anche come possiamo calcolare le probabilità congiunte (definiamo $L_tot = L1(1+s1)(1+s12) + L2(1+s2)$):
$$
\begin{align}
& P\big(A(\tau) > L1(1+s1) \cap A(2*\tau) >= L_{tot}\big) = \\
& \sum_i P\big(A(2*\tau) >= L_{tot}\big | A(\tau) \in [X_i, X_{i+1}])P\big(A(\tau) \in [X_i, X_{i+1}]\big)
\end{align}
$$
dove $[X_i, X_{i+1}]$ sono partizioni disgiunte (infinitesimali) di $A(\tau$) con $x_0 > L1(1+s1)$. Nel continuo:
$$
\int_{L1(1+s1)}^\infty dS_1 \bigg[ f(S_1) \int_{L_{tot}}^\infty f(S_2 | S(\tau) = S_1) dS_2\bigg]
$$
che si può scrivere come:
$$
\int_{L1(1+s1)}^\infty dS_1 \bigg[ f(S_1) N\big(d2=\frac{ln(S_1/L_{tot}) - 0.5\sigma^2\tau}{\sigma \sqrt{\tau}}\big)\bigg]
$$
dove f(S_1) è la distribuzione lognormale:
$$
f(S_1;\mu = \ln A(0) - 0.5\sigma^2\tau,\sigma \sqrt{\tau}) = \frac{1}{ S_1\sigma \sqrt{2 \pi \tau}}\exp\left[-\frac {(\ln S_1 - \mu)^{2}} {2\sigma^{2}\tau}\right]
$$
Conviene anche scrivere la probabilità per il mdesimo evento nella misura indotta dall'asset:
$$
\int_{L1(1+s1)}^\infty dS_1 \bigg[ f'(S_1) N\big(d1=\frac{ln(S_1/L_{tot}) + 0.5\sigma^2\tau}{\sigma \sqrt{\tau}}\big)\bigg]
$$
con
$$
f'(S_1;\mu = \ln A(0) + 0.5\sigma^2\tau,\sigma \sqrt{\tau}) = \frac{1}{ S_1\sigma \sqrt{2 \pi \tau}}\exp\left[-\frac {(\ln S_1 - \mu)^{2}} {2\sigma^{2}\tau}\right]
$$
Per quanto riguarda la probabilità dell'evento $$A(\tau) > L1(1+s1) \cap A(2*\tau) < L_{tot}$$ basta sostituire nelle precedenti formule $d1$ e $d2$ con $-d1$ e $-d2$.
Cerchiamo di non perdere l'obbiettivo: tutti questi conti servono per determinare s1, s2, s12 in modo che si possa stimare la variazione che l'equity subisce se al tempo $t = 0^+$ viene finanziato l'acquisto di cassa mediante debito senior. Contrariamente al caso uniperiodale, in questo caso ci sono infiniti modi per finanziare l'acquisto, ovvero una qualsiasi combinazione lineare di debiti a scadenze $\tau$ e $2\tau$. Intanto vediamo qual è la formula per valutare l'equity prima della nuova operazione:
$$
E(0) = \E\big[\1\big\{A(\tau) > L1(1+s1) \cap A(2*\tau) >= L_{tot}\big\} (A(2\tau) - L_{tot})\big]
$$
Se assumiamo che:
- la cassa viene finanziata emettendo debito con la stessa prorpozionalità di L1, L2
- gli spread s1, s2, s12, rimangono invariati (C << A)
è ragionevole ipotizzare che il nuovo valore dell'equity sia:
$$
E(0) = \E\big[\1\big\{(A(\tau) + C) > (L1 + C\frac{L1}{L1+L2})(1+s1) \cap (A(2*\tau) + C) >= L'_{tot}\big\} (A(2\tau) + C - L'_{tot})\big]
$$
dove
$$
L'_{tot} = (L1+C\frac{L1}{L1+L2})(1+s1)(1+s12) + (L2+C\frac{L2}{L1+L2})(1+s2)
$$

Adesso al lavoro con i conti!

In [1]:
import math
import numpy as np
import scipy as sp
import scipy.stats
import scipy.optimize
from scipy.integrate import quad


class Integrand(object):
    def __init__(self, S0, tau, vol, Ltot, isRiskNeutral, parity):
        self.tau = tau
        self.vol = vol
        self.Ltot = Ltot
        if isRiskNeutral:
            self.measuresign = -1
        else:
            self.measuresign = 1
        self.parity = parity
        self.S0 = S0
        self.mu = math.log(S0) + self.measuresign * 0.5 * vol * vol * tau
        self.var = vol * vol * tau

    def integrand(self, S):
        pdf = math.exp(-0.5 * ((math.log(S) - self.mu) ** 2) / self.var)
        pdf = pdf / (S * math.sqrt(2 * math.pi * self.var))
        d = self.parity * (math.log(S / self.Ltot) + 0.5 * self.measuresign * self.var) / (
        self.vol * math.sqrt(self.tau))
        N = sp.stats.norm.cdf(d)
        return pdf * N

    def integrandtest(self, S):
        pdf = math.exp(-0.5 * ((math.log(S) - self.mu) ** 2) / self.var)
        pdf = pdf / (S * math.sqrt(2 * math.pi * self.var))
        pdf = pdf * max(S - self.S0, 0.)
        return pdf

    @staticmethod
    def test():
        integrand = Integrand(100, 1, 0.2, 100, True, 1)
        print integrand.integrand(100)

    @staticmethod
    def testcall():
        integrand = Integrand(100, 1, 0.2, 100, True, 1)
        print quad(integrand.integrandtest, 0, 300)[0]


def jointprob(S0, tau, vol, L1, Ltot, isRiskNeutral, parity):
    integrand = Integrand(S0, tau, vol, Ltot, isRiskNeutral, parity)
    return quad(integrand.integrand, L1, 300)[0]


def ts_L1(A0, tau, vol, L1, L2, s1):
    d2 = (math.log(A0 / (L1*(1+s1))) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    d1 = (math.log(A0 / (L1*(1+s1))) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    Nd1 = sp.stats.norm.cdf(-d1)
    return L1 * (1 + s1) * Nd2 + float(L1) / float(L1 + L2) * A0 * Nd1


def ts_L12(A0, tau, vol, L1, L2, s1, s2, s12):
    d2 = (math.log(A0 / (L1*(1+s1))) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    Ltot = L1 * (1 + s1) * (1 + s12) + L2 * (1 + s2)
    defconditionalprob = jointprob(A0, tau, vol, L1 * (1 + s1), Ltot, False, -1) / Nd2
    survconditionalprob = jointprob(A0, tau, vol, L1 * (1 + s1), Ltot, True, 1) / Nd2
    L12 = float(L1) / float(L1 + L2) * A0 * defconditionalprob + float(L1) * (1 + s1) * (1 + s12) * survconditionalprob
    return L12


def ts_L2(A0, tau, vol, L1, L2, s1, s2, s12):
    d1 = (math.log(A0 / (L1*(1+s1))) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(-d1)
    Ltot = L1 * (1 + s1) * (1 + s12) + L2 * (1 + s2)
    defjointprob = jointprob(A0, tau, vol, L1 * (1 + s1), Ltot, False, -1)
    survjointprob = jointprob(A0, tau, vol, L1 * (1 + s1), Ltot, True, 1)
    L2_res = float(L2) / float(L1 + L2) * A0 * (Nd1 + defjointprob)
    L2_res = L2_res + float(L2) * (1 + s2) * survjointprob
    return L2_res


def ts_s1(A, tau, vol, L1, L2):
    res = sp.optimize.minimize(
        lambda s1: (ts_L1(A, tau, vol, L1, L2, s1) - L1) ** 2,
        [0.0],
        bounds=[(-0.1, 1.0)])
    return res.x[0]


def ts_s12_s2(A, tau, vol, L1, L2, s1):
    res = sp.optimize.minimize(
        lambda x: ((ts_L12(A, tau, vol, L1, L2, s1, x[0], x[1]) - L1 * (1 + s1))** 2 +
                   (ts_L2(A, tau, vol, L1, L2, s1, x[0], x[1]) - L2) ** 2),
        [0.0, 0.0],
        bounds=[(0.00001, 0.2), (0.00001, 0.2)])
    return res.x

def ts_equity(A0, tau, vol, L1, L2, s1, s2, s12, C):
    L1C = L1 + float(L1) / float(L1 + L2) * C
    L2C = L2 + float(L2) / float(L1 + L2) * C
    Ltot = L1C * (1 + s1) * (1 + s12) + L2C * (1 + s2)
    survjointprob_rn = jointprob(A0, tau, vol, L1C * (1 + s1) - C, Ltot - C, True, 1)
    survjointprob_asset = jointprob(A0, tau, vol, L1C * (1 + s1) - C, Ltot - C, False, 1)
    return A0 * survjointprob_asset - (Ltot - C) * survjointprob_rn
    

A = 100.0
L1 = 89.99
L2 = 90. - L1
vol = 0.1
tau = 1.0
C = 20

s1 = ts_s1(A, tau, vol, L1, L2)
s2, s12 = ts_s12_s2(A, tau, vol, L1, L2, s1)
e = ts_equity(A, tau, vol, L1, L2, s1, s2, s12, 0)
eC = ts_equity(A, tau, vol, L1, L2, s1, s2, s12, C)
print "s1: ", s1, "s2: ", s2, "s12: ", s12
print "equity: ", e
print "equity C: ", eC

s1:  0.00953196687802 s2:  1.01063037452e-05 s12:  0.0131050768842
equity:  10.0001486197
equity C:  9.69020959283


Questo ultimo risultato numerico fa capire che c'è qualcosa che non va nella formula per l'equity: il problema è che in $\tau$ il finanziatore a breve deve recepire il cambiamento di portafoglio (adesso c'è $C$) ed applicare uno spread differente da quello forward $s12$. Se non c'è una formula analitica si deve andare di Montecarlo.
$$
E(0) = \E\big[\E\big[\max\big (A(2\tau) + C - L2'(1+s2) - L1'(1+s1)(1+\tilde{s}12), 0\big) |\mathcal{F}_\tau\big]\big|\mathcal{F}_0\big]
$$
dove con l'apice si intende il finanziamento incrementato per la quota parte di $C$ e $\tilde{s}12$ è lo spread, ignoto in $t=0$, che soddisfa l'equazione:
$$
L1'(1+s1) = \E\big[\1\{A(2\tau) + C \geq L2'(1+s2) + L1'(1+s1)(1+\tilde{s}12)\}L1(1+s1)(1+\tilde{s}12)+A(2\tau)\frac{L1'}{L_{tot}'}\1\{--<--\}\big]
$$


In [2]:
def ts_Ltilde12(A0, C, tau, vol, L1, L2, s1, s2, s12):
    L1C = L1 + float(L1) / float(L1 + L2) * C
    L2C = L2 + float(L2) / float(L1 + L2) * C
    Ltot = L1C * (1 + s1) * (1 + s12) + L2C * (1 + s2)
    d2 = (math.log(A0 / (Ltot - C)) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    survival_price = Nd2 * L1C*(1+s12)
    d1 = (math.log(A0 / (Ltot - C)) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(-d1)
    recovery_price =  float(L1C) / float(L1C + L2C) * (A0 * Nd1 + C*(1-Nd2))
    return survival_price + recovery_price

def ts_stilde12(A0, C, tau, vol, L1, L2, s1, s2):
    L1C = L1 + float(L1) / float(L1 + L2) * C
    res = sp.optimize.minimize(
    lambda s1: (ts_Ltilde12(A0, C, tau, vol, L1, L2, s1, s2, s12) - L1C*(1+s1)) ** 2,
    [0.0],
    bounds=[(-0.1, 0.2)])
    return res.x[0]

def ts_pathGen(runs, A0, tau, vol):
    np.random.seed(1)
    nsteps = 1
    vol2 = vol * vol
    epsilon = np.random.normal(0, 1, runs)
    paths = np.ones(2 * runs)
    paths[0:runs] = A0 * np.exp(-0.5*vol2 * tau + vol * math.sqrt(tau) * epsilon[:])
    paths[runs:2*runs] = A0 * np.exp(-0.5*vol2 * tau - vol * math.sqrt(tau) * epsilon[:])
    return paths

def ts_E_MC(A0, C, tau, vol, L1, L2, nruns):
    # First we find s1, s2, s12 without C
    s1 = ts_s1(A, tau, vol, L1, L2)
    s2, s12 = ts_s12_s2(A, tau, vol, L1, L2, s1)
    print s1, s2, s12
    
    # Now the montecarlo
    paths = ts_pathGen(nruns, A0, tau, vol)
    equities = np.zeros(2 * nruns)
    L1C = L1 + float(L1) / float(L1 + L2) * C
    L2C = L2 + float(L2) / float(L1 + L2) * C
    s = []
    for i in range(2 * nruns):
        if paths[i] + C > L1C*(1+s1):
            stilde12 = ts_stilde12(paths[i], C, tau, vol, L1, L2, s1, s2)
            s.append(stilde12)
            Ltot = L1C * (1 + s1) * (1 + stilde12) + L2C * (1 + s2)
            d2 = (math.log(paths[i] / (Ltot - C)) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
            Nd2 = sp.stats.norm.cdf(d2)
            d1 = (math.log(paths[i] / (Ltot - C)) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
            Nd1 = sp.stats.norm.cdf(d1)
            equities[i] = paths[i] * Nd1 - (Ltot - C) * Nd2
        else:
            equities[i] = 0.
    print np.mean(s)
    return np.mean(equities)


A = 100.0
L1 = 89.99
L2 = 90. - L1
vol = 0.1
tau = 1.0
C = 0.
runs = 10000
print ts_E_MC(A, C, tau, vol, L1, L2, runs)


    

0.00953196687802 1.01063037452e-05 0.0131050768842
0.0018861325695
10.5670390916


Sopra c'è grossa crisi, così tanta che passo al KVA!

KVA and friends

Che cosa è il KVA in questo modello? Come per l'FVA si definisce il KVA come la variazione di equity prima e dopo aver rispettato gli obblighi normativi. In generale FVA e KVA saranno calcolati insieme considerando l'equity prima e dopo un investimento in un attivo.

Partiamo dalle usuali condizioni di una banca con un attivo $A$, un debito $L$ appena rinnovato su cui viene pagato un fair spread $s$ (quindi equity $E=A-L$). Supponiamo per semplicità che l'attivo sia composto da un unico investimento con un risk weight da normativa basilea/CRD IV $w$. Perché la banca possa condurre la sua attività deve necessariamente essere che 
$$
\frac{E}{wA} \geq x\%
$$
dove $x\%$ è un valore regolamentare. Stiamo implicitamente assumendo in questo modello che il market cap $E$ sia esattamente uguale al CET1 della normativa. Se adesso la banca intende investire tramite finanziamento in un altro asset rischioso, il vincolo normativo deve essere rispettato. Se si assume che il finanziatore è a conoscenza del requisito normativo, devono valere contemporaneamente le seguenti equazioni:
$$
\begin{align}
\frac{E_1}{w(1+\alpha)A+w_1A_1} & \geq x\% \\
L_1 & <= \E\big[\1\{default\}L_1(1+s1) + \frac{L_1}{L_1 + L}\big((1+\alpha)A + A_1\big)\1\{not default\}\big]
\end{align}
$$
dove con $default$ si intende l'evento:
$$
(1+\alpha)A + A_1 \geq L(1+s) + L_1(1+s1)
$$
Due parole sulle formule precedenti: si suppone che a seguito dell'investimento in $A1$ la banca sia costretta ad effettuare un aumento di capitale ed il capitale verrà investito interamente nell'attivo esistente $A$, che equivale ad ipotizzare una continuità di business; l'aumento di capitale è quindi pari a $\alpha A$. L'investitore onniscente conosce quello che sarà il bilancio a seguito dell'operazione nel suo complesso (acquisto di $A1$ e aumento di capitale) e vuole quindi essere remunerato con un spread che soddisfa la seconda diseguaglianza. Con queste due diseguaglianze è possibile determinare le coppie $\alpha$, $s1$ ammissibili.

Proviamo a risolvere il problema in un caso semplice, quello in cui $w=1$ e $A_1 = \beta A$. Semplicemente la banca aumenta la sua esposizione nell'attivo. In questo caso ci si aspetta intuitivamente che il modello fornica un FVA-KVA nullo: è come aumentare proporzionalmente attivo e passivo della azienda senza alterare la leva finanziaria. Verifichiamo con le formule sopra che in questo caso divengono (le disequazioni sono diventate equazioni):
$$
\begin{align}
\frac{E_1}{(1+\alpha +\beta)A} & = x\% \\
L_1 & = \E\big[\1\{default\}L_1(1+s1) + \frac{L_1}{L_1 + L}(1+\alpha + \beta)A)\1\{not default\}\big]
\end{align}
$$
dove con $default$ si intende l'evento:
$$
(1+\alpha +\beta)A \geq L(1+s) + L_1(1+s1)
$$
L'equity $E_1$ deve chiaramente soddisfare a:
$$
E_1 = \E\big[ \max\big((1+\alpha+\beta)A - L(1+s)-L_1(1+s1), 0 \big) \big]
$$

In [2]:
import math
import numpy as np
import scipy as sp
import scipy.stats
import scipy.optimize
from scipy.integrate import quad

def KVA_ratio(E, A, alpha, beta):
    ratio =  E / (A * (1 + alpha + beta))
    return ratio

def KVA_E(A, vol, tau, L, s):
    Ltot = L * (1 + s)
    d2 = (math.log(A / Ltot) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    d1 = (math.log(A / Ltot) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    price = A * Nd1 - Ltot * Nd2
    return price


def KVA_s(A, vol, tau, L):
    res = sp.optimize.minimize(
        lambda x: (KVA_E(A, vol, tau, L, x) - (A - L)) ** 2,
        [0.0],
        bounds=[(0.00001, 0.2)], tol=1e-15)
    return res.x[0]


def KVA_E1(A, vol, tau, L, s, L1, s1, alpha, beta):
    Ltot = L * (1 + s) + L1 * (1 + s1)
    A0 = A * (1 + alpha + beta)
    d2 = (math.log(A0 / Ltot) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    d1 = (math.log(A0 / Ltot) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    price = A0 * Nd1 - Ltot * Nd2
    return price


def KVA_L1(A, vol, tau, L, s, L1, s1, alpha, beta):
    Ltot = L * (1 + s) + L1 * (1 + s1)
    A0 = A * (1 + alpha + beta)
    d2 = (math.log(A0 / Ltot) - 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd2 = sp.stats.norm.cdf(d2)
    survival_price = Nd2 * L1 * (1 + s1)
    d1 = (math.log(A0 / Ltot) + 0.5 * vol * vol * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(-d1)
    recovery_price = float(L1) / float(L1 + L) * A0 * Nd1
    return survival_price + recovery_price


def KVA_findparams(A, vol, tau, L, s, L1, ratio):
    alpha = float(L1) / float(A)
    res = sp.optimize.minimize(
        lambda x: ((KVA_ratio(KVA_E1(A, vol, tau, L, s, L1, x[0], alpha, x[1]), A, alpha, x[1]) - ratio) ** 2 +
                   (KVA_L1(A, vol, tau, L, s, L1, x[0], alpha, x[1]) - L1) ** 2),
        [0.0, 0.0],
        bounds=[(0.00001, 0.2), (0.00001, 0.1)],
        tol=1e-15)
    return res.x


A = 100.0
L = 90.0
vol = 0.2
tau = 1.0
x = 0.1
s = KVA_s(A, vol, tau, L)
print s
L1 = 10
alpha = L1 / A
s1, beta = KVA_findparams(A, vol, tau, L, s, L1, x)
E1 = KVA_E1(A, vol, tau, L, s, L1, s1, alpha, beta)
print s1, beta, E1
print "KVA: ", E1 - beta * A - (A-L)


0.0660052969378
0.0660052177493 0.0111112245383 11.1111188521
KVA:  -3.6017134839e-06


CVD! Il KVA è nullo. Ci sono altri due casi interessanti, sempre a parità di risk weight $w$, quando $A_1$ ha una volatilità maggiore e quando ce l'ha minore. Il problema di questi casi è che il modello diventa matematicamente complicato... Ma non troppo; alla fine dovrebbe essere risolvibile con integrazioni numeriche bidimensionali.
Le equazioni da risolvere contemporaneamente sono:
$$
\begin{align}
\frac{\E\big[ \max\big((1+\alpha)A + A_1 - L(1+s)-L_1(1+s1), 0 \big) \big]}{w(1+\alpha)A+w_1A_1} & = x\% \\
\E\big[\1\{notdefault\}L_1(1+s1) + \frac{L_1}{L_1 + L}\big((1+\alpha)A + A_1\big)\1\{default\}\big] & = L
\end{align}
$$
dove con $default$ si intende l'evento:
$$
(1+\alpha)A + A_1 \geq L(1+s) + L_1(1+s1)
$$


In [8]:
import math
import numpy as np
import scipy as sp
import scipy.stats
import scipy.optimize
from scipy.integrate import quad

class BiDimensionalCall(object):
    def __init__(self, A, B, K, vols, rho, t):
        self.A = A * math.exp(-0.5 * vols[0] * vols[0] * t)
        self.B = B * math.exp(-0.5 * vols[1] * vols[1] * t)
        self.vols = np.array(vols) * math.sqrt(t)
        self.rho = rho
        self.a00 = 1. / (self.vols[0] * self.vols[0] * (1 - rho * rho))
        self.a11 = 1. / (self.vols[1] * self.vols[1] * (1 - rho * rho))
        self.a01 = - rho / ((1. - rho * rho) * self.vols[0] * self.vols[1])
        self.K = K
        self.sigmabar = self.vols[0] * math.sqrt(1 - rho * rho)
        self.t = t

    def xa_bar(self, xa):
        first = self.A * math.exp(xa)
        second = self.B * math.exp(self.xb)
        return first + second - self.K

    def integrand(self, xb):
        self.xb = xb
        xabar = scipy.optimize.newton(self.xa_bar, 0.)
        mu = - self.a01 * xb / self.a00
        d1 = (xabar - mu) / self.sigmabar - self.sigmabar
        d2 = (xabar - mu) / self.sigmabar
        first = self.A * math.exp(mu + 0.5 * self.sigmabar * self.sigmabar) * sp.stats.norm.cdf(-d1)
        second = self.B * math.exp(xb) * sp.stats.norm.cdf(-d2)
        third = self.K * sp.stats.norm.cdf(-d2)
        pdf = math.exp(-0.5 * self.a11 * xb * xb + 0.5 * self.a01 * self.a01 / self.a00 * xb * xb) / (math.sqrt(2 * math.pi) * self.vols[1])
        summation = first + second - third
        return summation * pdf

    def prob_surv_integrand(self, xb):
        self.xb = xb
        xabar = scipy.optimize.newton(self.xa_bar, 0.)
        mu = - self.a01 * xb / self.a00
        d2 = (xabar - mu) / self.sigmabar
        third = sp.stats.norm.cdf(-d2)
        pdf = math.exp(-0.5 * self.a11 * xb * xb + 0.5 * self.a01 * self.a01 / self.a00 * xb * xb) / (math.sqrt(2 * math.pi) * self.vols[1])
        return third * pdf

    def prob_surv(self):
        lbound = -6. * self.vols[1]
        ubound = 6. * self.vols[1]
        return scipy.integrate.quad(self.prob_surv_integrand, lbound, ubound)[0]

    def prob_def_integrand(self, xb):
        self.xb = xb
        xabar = scipy.optimize.newton(self.xa_bar, 0.)
        mu = - self.a01 * xb / self.a00
        d1 = (xabar - mu) / self.sigmabar - self.sigmabar
        d2 = (xabar - mu) / self.sigmabar
        first = self.A * math.exp(mu + 0.5 * self.sigmabar * self.sigmabar) * sp.stats.norm.cdf(d1)
        second = self.B * math.exp(xb) * sp.stats.norm.cdf(d2)
        pdf = math.exp(-0.5 * self.a11 * xb * xb + 0.5 * self.a01 * self.a01 / self.a00 * xb * xb) / (math.sqrt(2 * math.pi) * self.vols[1])
        return (first + second) * pdf

    def prob_def(self):
        lbound = -6. * self.vols[1]
        ubound = 6. * self.vols[1]
        return scipy.integrate.quad(self.prob_def_integrand, lbound, ubound)[0]

    def price(self):
        lbound = -6. * self.vols[1]
        ubound = 6. * self.vols[1]
        return scipy.integrate.quad(self.integrand, lbound, ubound)[0]


def KVA2d_E1(A, vol, A1, vol1, rho, tau, L, s, L1, s1, alpha):
    call = BiDimensionalCall(A * (1 + alpha), A1, L * (1 + s) + L1 * (1 + s1), [vol, vol1], rho, tau)
    return call.price()


def KVA2d_L1(A, vol, A1, vol1, rho, tau, L, s, L1, s1, alpha):
    call = BiDimensionalCall(A * (1 + alpha), A1, float(L) * (1 + s) + float(L1) * (1 + s1), [vol, vol1], rho, tau)
    probdef = call.prob_def()
    default_price = call.prob_def() * float(L1) / (float(L1) + float(L))
    probsurv = call.prob_surv()
    survival_price = probsurv * float(L1) * (1 + s1)
    return default_price + survival_price

def KVA2d_ratio(E, A, w, A1, w1, alpha):
    ratio =  E / (w * A * (1 + alpha) + w1 * A1)
    return ratio


def KVA2D_findparams(A, w, vol, A1, w1, vol1, rho, tau, L, s, L1,ratio):
    res = sp.optimize.minimize(
        lambda x: ((KVA2d_ratio(KVA2d_E1(A, vol, A1, vol1, rho, tau, L, s, L1, x[0], x[1]), A, w, A1, w1, x[1]) - ratio) ** 2 +
                   (KVA2d_L1(A, vol, A1, vol1, rho, tau, L, s, L1, x[0], x[1]) - L1) ** 2),
        [0.0, 0.0],
        bounds=[(0.00001, 0.2), (0.00001, 0.1)],
        tol=1e-15)
    return res.x

A = 100.0
L = 90.0
vol = 0.2
tau = 1.0
x = 0.1
s = KVA_s(A, vol, tau, L)
print s
L1 = 10.
alpha = L1 / A
s1, beta = KVA_findparams(A, vol, tau, L, s, L1, x)
E1 = KVA_E1(A, vol, tau, L, s, L1, s1, alpha, beta)
print s1, beta, E1
print "vol = vol1 (same w) -> KVA: ", E1 - beta * A - (A-L)

A1 = 10.
vol1 = 0.4
rho = 0.9999999
alpha = beta
w = 1.
w1 = 1.
s1, alpha = KVA2D_findparams(A, w, vol, A1, w1, vol1, rho, tau, L, s, L1, x)
print "s1: ", s1, ", alpha: ", alpha
E1 = KVA2d_E1(A, vol, A1, vol1, rho, tau, L, s, L1, s1, alpha)
E = A - L
KVA = E1 - (A*(1+alpha) - L)
print "vol1 > vol -> KVA-FVA: ", KVA

vol1 = 0.1
s1, alpha = KVA2D_findparams(A, w, vol, A1, w1, vol1, rho, tau, L, s, L1, x)
print "s1: ", s1, ", alpha: ", alpha
E1 = KVA2d_E1(A, vol, A1, vol1, rho, tau, L, s, L1, s1, alpha)
E = A - L
KVA = E1 - (A*(1+alpha) - L)
print "vol1 < vol -> KVA-FVA: ", KVA

vol1 = 0.000001
w1 = 0.
s1, alpha = KVA2D_findparams(A, w, vol, A1, w1, vol1, rho, tau, L, s, L1, x)
print "s1: ", s1, ", alpha: ", alpha
E1 = KVA2d_E1(A, vol, A1, vol1, rho, tau, L, s, L1, s1, alpha)
E = A - L
KVA = E1 - (A*(1+alpha) - L)
print "vol1 = 0., w1 = 0. -> KVA-FVA: ", KVA
print "ratio: ", KVA2d_ratio(E1, A, w, A1, w1, alpha)


0.0660052969378
0.0660052177493 0.0111112245383 11.1111188521
vol = vol1 (same w) -> KVA:  -3.6017134839e-06
s1:  0.0893251568764 , alpha:  1e-05
vol1 > vol -> KVA-FVA:  1.06440313688
s1:  0.0551158319144 , alpha:  0.0172202080668
vol1 < vol -> KVA-FVA:  -0.549810760755
s1:  0.0553706252007 , alpha:  0.00577138494098
vol1 = 0., w1 = 0. -> KVA-FVA:  -0.519418911881
ratio:  0.100000056999
